## 参考文献：
pickle使用: http://www.cnblogs.com/pzxbc/archive/2012/03/18/2404715.html

sklearn: http://www.cnblogs.com/jasonfreak/p/5448462.html

sklearn官方文档：http://sklearn.lzjqsdd.com/modules/decomposition.html#decompositions

**数据和特征决定了机器学习的上限，而模型和算法只是逼近这个上限而已。**

---
# 作业相关

个人理解：
1. 由于数据没给特征矩阵，所以要先自己根据数据构造特征矩阵
2. 没有缺失值，维数不大，所以基本不需要降维，规格化也不用，因为特征值都是我们生成的，但预处理需要处理noise之类
3. 关键还是特征要选得好

### 1.测试过的特征

1. 元音个数 *
2. 前缀 *
3. 后缀
4. 音标个数
5. 单词长度
6. 元音序列 *
7. 音标序列

*加星号的效果比较好*

In [12]:

import pandas as pd
import numpy as np
import sklearn
import sys

sys.path.append('..')

import helper

import matplotlib.pyplot as plt

testing_data = helper.read_data('../asset/tiny_test.txt')

#定义元辅音集合
vowel = "AA, AE, AH, AO, AW, AY, EH, ER, EY, IH, IY, OW, OY, UH, UW".replace(",","").split()
consonant = "P, B, CH, D, DH, F, G, HH, JH, K, L, M, N, NG, R, S, SH, T, TH, V, W, Y, Z, ZH".replace(",","").split()

#元、辅音合集合
vowelAndconsonan = vowel + consonant;

In [14]:
#这里是pd试验田
# df_train = pd.read_csv('../asset/pd_TrainData.txt', sep=':')
# df_train.head()

In [15]:
#获取前缀后缀, 直接用了别人的
def s_has_pre(s):
    pre = "an,dis,in,ig,il,im,ir,ne,n,non,neg,un,male,mal,pseudo,mis,de\
    un,anti,ant,contra,contre,contro,counter,ob,oc,of,op,with,by,circum,\
    circu,de,en,ex,ec,es,fore,in,il,im,ir,inter,intel,intro,medi,med,mid,out,\
    over,post,pre,pro,sub,suc,suf,sug,sup,sur,sus,sur,trans,under,up,\
    ante,anti,ex,fore,mid,medi,post,pre,pri,out,over,post,pre,pro,sub,suc,suf,\
    sug,sum,sup,sur,sus,super,sur,trans,under,up,ante,anti,ex,fore,mid,medi,post,\
    pre,pri,pro,re,by,extra,hyper,out,over,sub,suc,sur,super,sur,under,vice,com,\
    cop,con,cor,co,syn,syl,sym,al,over,pan,ex,for,re,se,dia,per,pel,trans，ad,\
    ac,af,ag,an,ap,ar,as,at,ambi,bin,di,twi,tri,thir,deca,deco,dec,deci,hecto,\
    hect,centi,kilo,myria,mega,micro,multi,poly,hemi,demi,semi,pene,arch,auto,bene,\
    eu,male,mal,macro,magni,micro,aud,bio,ge,phon,tele,\
    ac,ad,af,ag,al,an,ap,as,at,an,ab,abs,acer,acid,acri,act,ag,acu,aer,aero,ag,agi,\
    ig,act,agri,agro,alb,albo,ali,allo,alter,alt,am,ami,amor,ambi,ambul,ana,ano,andr,\
    andro,ang,anim,ann,annu,enni,ante,anthrop,anti,ant,anti,antico,apo,ap,aph,aqu,arch,\
    aster,astr,auc,aug,aut,aud,audi,aur,aus,aug,auc,aut,auto,bar,be,belli,bene,bi,bine,\
    bibl,bibli,biblio,bio,bi,brev,cad,cap,cas,ceiv,cept,capt,cid,cip,cad,cas,calor,capit,\
    capt,carn,cat,cata,cath,caus,caut,cause,cuse,cus,ceas,ced,cede,ceed,cess,cent,centr,\
    centri,chrom,chron,cide,cis,cise,circum,cit,civ,clam,claim,clin,clud,clusclaus,co,cog,\
    col,coll,con,com,cor,cogn,gnos,com,con,contr,contra,counter,cord,cor,cardi,corp,cort,\
    cosm,cour,cur,curr,curs,crat,cracy,cre,cresc,cret,crease,crea,cred,cresc,cret,crease,\
    cru,crit,cur,curs,cura,cycl,cyclo,de,dec,deca,dec,dign,dei,div,dem,demo,dent,dont,derm,\
    di,dy,dia,dic,dict,dit,dis,dif,dit,doc,doct,domin,don,dorm,dox,duc,duct,dura,dynam,dys,\
    ec,eco,ecto,en,em,end,epi,equi,erg,ev,et,ex,exter,extra,extro,fa,fess,fac,fact,fec,fect,\
    fic,fas,fea,fall,fals,femto,fer,fic,feign,fain,fit,feat,fid,fid,fide,feder,fig,fila,fili,\
    fin,fix,flex,flect,flict,flu,fluc,fluv,flux,for,fore,forc,fort,form,fract,frag,frai,fuge,\
    fuse,gam,gastr,gastro,gen,gen,geo,germ,gest,giga,gin,gloss,glot,glu,glo,gor,grad,gress\
    ,gree,graph,gram,graf,grat,grav,greg,hale,heal,helio,hema,hemo,her,here,hes,hetero,hex\
    ,ses,sex,homo,hum,human,hydr,hydra,hydro,hyper,hypn,an,ics,ignis,in,im,in,im,il,ir,infra\
    ,inter,intra,intro,ty,jac,ject,join,junct,judice,jug,junct,just,juven,labor,lau,lav,lot\
    ,lut,lect,leg,lig,leg,levi,lex,leag,leg,liber,liver,lide,liter,loc,loco,log,logo,ology\
    ,loqu,locut,luc,lum,lun,lus,lust,lude,macr,macer,magn,main,mal,man,manu,mand,mania,mar\
    ,mari,mer,matri,medi,mega,mem,ment,meso,meta,meter,metr,micro,migra,mill,kilo,milli,min\
    ,mis,mit,miss,mob,mov,mot,mon,mono,mor,mort,morph,multi,nano,nasc,nat,gnant,nai,nat,nasc\
    ,neo,neur,nom,nom,nym,nomen,nomin,non,non,nov,nox,noc,numer,numisma,ob,oc,of,op,oct,oligo\
    ,omni,onym,oper,ortho,over,pac,pair,pare,paleo,pan,para,pat,pass,path,pater,patr,path,pathy\
    ,ped,pod,pedo,pel,puls,pend,pens,pond,per,peri,phage,phan,phas,phen,fan,phant,fant,phe,phil\
    ,phlegma,phobia,phobos,phon,phot,photo,pico,pict,plac,plais,pli,ply,plore,plu,plur,plus,pneuma\
    ,pneumon,pod,poli,poly,pon,pos,pound,pop,port,portion,post,pot,pre,pur,prehendere,prin,prim,\
    prime,pro,proto,psych,punct,pute,quat,quad,quint,penta,quip,quir,quis,quest,quer,re,reg,recti\
    ,retro,ri,ridi,risi,rog,roga,rupt,sacr,sanc,secr,salv,salu,sanct,sat,satis,sci,scio,scientia,\
    scope,scrib,script,se,sect,sec,sed,sess,sid,semi,sen,scen,sent,sens,sept,sequ,secu,sue,serv,\
    sign,signi,simil,simul,sist,sta,stit,soci,sol,solus,solv,solu,solut,somn,soph,spec,spect,spi,\
    spic,sper,sphere,spir,stand,stant,stab,stat,stan,sti,sta,st,stead,strain,strict,string,stige,\
    stru,struct,stroy,stry,sub,suc,suf,sup,sur,sus,sume,sump,super,supra,syn,sym,tact,tang,tag,tig,\
    ting,tain,ten,tent,tin,tect,teg,tele,tem,tempo,ten,tin,tain,tend,tent,tens,tera,term,terr,terra,\
    test,the,theo,therm,thesis,thet,tire,tom,tor,tors,tort,tox,tract,tra,trai,treat,trans,tri,trib,\
    tribute,turbo,typ,ultima,umber,umbraticum,un,uni,vac,vade,vale,vali,valu,veh,vect,ven,vent,ver,\
    veri,verb,verv,vert,vers,vi,vic,vicis,vict,vinc,vid,vis,viv,vita,vivi,voc,voke,vol,volcan,volv\
    ,volt,vol,vor,with,zo".replace(" ","").split(",")
    for i in pre:
        if s.startswith(i.upper()):
            return 1
    return 0

# 验证无用
def s_has_end(s):
    end = "ee,ese,esque,se,eer,ique,ty,less,ness,ly,ible,able,ion,ic,ical,al,ian,ic,\
    ion,ity,ment,ed,es,er,est,or,ary,ory,ous,cy,ry,ty,al,ure,ute,ble,ar,ly,less,ful,ing,\
    ,inal,tion,sion,osis,oon,sce,\
    que,ette,eer,ee,aire,able,ible,acy,cy,ade,age,al,al,ial,ical,an,ance,ence,ancy,\
    ency,ant,ent,ant,ent,ient,ar,ary,ard,art,ate,ate,ate,ation,cade,drome,ed,ed,en,en,\
    ence,ency,er,ier,er,or,er,or,ery,es,ese,ies,es,ies,ess,est,iest,fold,ful,ful,fy,ia,\
    ian,iatry,ic,ic,ice,ify,ile,ing,ion,ish,ism,ist,ite,ity,ive,ive,ative,itive,ize,less,\
    ly,ment,ness,or,ory,ous,eous,ose,ious,ship,ster,ure,ward,wise,ize,phy,ogy,ity,ion,ic,ical,al".replace(" ","").split(",")
    for i in end:
        if s.endswith(i.upper()):
            return 1
    return 0

## 以下是特征提取的两个函数，因为train数据和test数据是不一样的，train数据有数字，所以给了两个特征提取函数

In [16]:
#输入训练单词和音标获取：目标值，单词长度，元音数, 音标序列，元音序列, 有无对应前缀，有无对应后缀
def getInfoOfPronsFromTrain(word, prons):
    wordLen = len(word)
    hasPre = s_has_pre(word)
    hasEnd = s_has_end(word)
    pronsSeq = []
    vowelsSeq = []
    stressVowelIndex = 0
    count = 0
    sub_prons = prons.split(' ')
    for i in range(len(sub_prons)):
        if sub_prons[i][-1].isdigit():
            vowelsSeq.append(vowel.index(sub_prons[i][0:-1])) #取得元音在元音集合中的序号
            pronsSeq.append(vowelAndconsonan.index(sub_prons[i][0:-1])) #取得元音在元、辅音集合中的序号
            count+=1
            if sub_prons[i][-1] == '1':
                stressVowelIndex = count
        else:
            pronsSeq.append(vowelAndconsonan.index(sub_prons[i])) #取得辅音在元辅、音集合中的序号
    return stressVowelIndex, wordLen, count, pronsSeq, vowelsSeq, hasPre, hasEnd

#输入测试单词和目标获取：元音数, 音标序列，元音序列, 有无对应前缀，有无对应后缀
def getInfoOfPronsFromTest(word, prons):
    hasPre = s_has_pre(word)
    hasEnd = s_has_end(word)
    pronsSeq = []
    vowelsSeq = []
    count = 0
    sub_prons = prons.split(' ')
    for i in range(len(sub_prons)):
        if sub_prons[i] in vowel:
            vowelsSeq.append(vowel.index(sub_prons[i])) #取得元音在元音集合中的序号
            count+=1
        pronsSeq.append(vowelAndconsonan.index(sub_prons[i])) #取得辅音在元辅、音集合中的序号
    return count, pronsSeq, vowelsSeq, hasPre, hasEnd

In [17]:
## function test 这里是我上面两个函数的测试
a1, a2 = training_data[3].split(':')

print(len(a1))

print(a1, a2)
b1, b2, b3, b4, b5, b6, b7= getInfoOfPronsFromTrain(a1, a2)
print(b1, b2, b3, b4, b5, b6, b7)

a1, a2 = testing_data[3].split(':')
print(a1, a2)
b1, b2, b3, b4, b5= getInfoOfPronsFromTest(a1, a2)
print(b1, b2, b3, b4, b5)

8
MUSCLING M AH1 S AH0 L IH0 NG
1 8 3 [26, 2, 30, 2, 25, 9, 28] [2, 2, 9] 0 1
LEARNING L ER N IH NG
2 [25, 7, 27, 9, 28] [7, 9] 0 1


In [18]:
targetArr = []
wordLenArr = []
countArr = []
pronsSeqArr = [] 
vowelsSeqArr = []
hasPreArr = []
hasEndArr = []

for i in range(len(training_data)):
    t_word, t_prons = training_data[i].split(':')
    t1, t2, t3, t4, t5, t6, t7 = getInfoOfPronsFromTrain(t_word, t_prons)
    targetArr.append(t1)
    wordLenArr.append(t2)
    countArr.append(t3)
    pronsSeqArr.append(t4) 
    vowelsSeqArr.append(t5)
    hasPreArr.append(t6)
    hasEndArr.append(t7)

In [22]:
df = pd.DataFrame({'data' : training_data,
                   'target' : targetArr,
                   'wordLen': wordLenArr,
                   'count' : countArr,
                   'pronsSeq' : pronsSeqArr,
                   'vowelsSeq' : vowelsSeqArr,
                   'hasPre' : hasPreArr,
                   'hasEnd' : hasEndArr
})


df.head()

,count,data,hasEnd,hasPre,pronsSeq,target,vowelsSeq,wordLen
0,2,COED:K OW1 EH2 D,1,1,"[24, 11, 6, 18]",1,"[11, 6]",4
1,2,PURVIEW:P ER1 V Y UW2,1,1,"[15, 7, 34, 36, 14]",1,"[7, 14]",7
2,2,HEHIR:HH EH1 HH IH0 R,1,0,"[22, 6, 22, 9, 29]",1,"[6, 9]",5
3,3,MUSCLING:M AH1 S AH0 L IH0 NG,1,0,"[26, 2, 30, 2, 25, 9, 28]",1,"[2, 2, 9]",8
4,4,NONPOISONOUS:N AA0 N P OY1 Z AH0 N AH0 S,1,1,"[27, 0, 27, 15, 12, 37, 2, 27, 2, 30]",2,"[0, 12, 2, 2]",12


In [23]:
df.describe()

,count,hasEnd,hasPre,target,wordLen
count,50000.000000,50000.0,50000.000000,50000.000000,50000.000000
mean,2.567340,1.0,0.443140,1.364080,7.582100
std,0.696358,0.0,0.496761,0.595326,1.879673
min,2.000000,1.0,0.000000,1.000000,1.000000
25%,2.000000,1.0,0.000000,1.000000,6.000000
50%,2.000000,1.0,0.000000,1.000000,7.000000
75%,3.000000,1.0,1.000000,2.000000,9.000000
max,4.000000,1.0,1.000000,4.000000,17.000000


In [24]:
#看看最长的音标数
maxlen = 0
for temp_arr in pronsSeqArr:
    if len(temp_arr) > maxlen:
        maxlen = len(temp_arr)
print('max length in pronsSeq', maxlen)

##看看最长的元音数
maxlen = 0
for temp_arr in vowelsSeqArr:
    if len(temp_arr) > maxlen:
        maxlen = len(temp_arr)
print('max length in vowelsSeq', maxlen)

max length in pronsSeq 14
max length in vowelsSeq 4


### 研究发现hasEnd的方差为0，舍去,  同时发现元音长度是一个很好的label

In [ ]:
df.groupby(['count', 'target']).count()

## 以下是特征矩阵构造
0: 只用元音个数

1: 元音个数+前缀

2: 元音个数+前缀+音节序列

3: 元音个数+前缀+元音序列

4: 单词长度+元音个数+前缀+元音序列

5: 元音个数+元音序列

In [ ]:
#构造特征矩阵0
labels_0 = []
for i in range(len(training_data)):
    labels_0.append([countArr[i]])

In [ ]:
#构造特征矩阵1
labels_1 = []
for i in range(len(training_data)):
    labels_1.append([countArr[i], hasPreArr[i]])

In [ ]:
pronsSeqArr2 = []

for temp in pronsSeqArr:
    k = 14 - len(temp)
    temp += [-1] * k
    pronsSeqArr2.append(temp)    

In [ ]:
#构造特征矩阵2
labels_2 = []
for i in range(len(training_data)):
    tempList = [countArr[i], hasPreArr[i]]
    tempList += pronsSeqArr2[i];
    labels_2.append(tempList)     

In [ ]:
labels_2;

In [ ]:
#构造特征矩阵3
vowelsSeqArr2 = []

for temp in vowelsSeqArr:
    k = 4 - len(temp)
    temp += [-1] * k
    vowelsSeqArr2.append(temp)   

labels_3 = []
for i in range(len(training_data)):
    tempList = [countArr[i], hasPreArr[i]]
    tempList += vowelsSeqArr2[i];
    labels_3.append(tempList) 



In [ ]:
labels_3;

In [ ]:
#构造特征矩阵4
labels_4  = []
for i in range(len(training_data)):
    tempList = [wordLenArr[i], countArr[i], hasPreArr[i]]
    tempList += vowelsSeqArr2[i];
    labels_4.append(tempList) 

In [ ]:
labels_4;

In [ ]:
#构造特征矩阵5  

labels_5 = []
for i in range(len(training_data)):
    tempList = [countArr[i]]
    tempList += vowelsSeqArr2[i];
    labels_5.append(tempList) 

In [ ]:
labels_5;

In [ ]:
df_labels_2 =  pd.DataFrame(labels_3)
df_labels_2.describe()

## test data太少，直接用train中的50000个进行交叉验证

In [ ]:
from sklearn import tree
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import f1_score 

x_train, x_test, y_train, y_test = train_test_split(labels_3, targetArr, test_size = 0.2) 

### 决策树

In [ ]:

#使用信息熵作为划分标准，对决策树进行训练
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(x_train, y_train)

#系数反映每个特征的影响力。越大表示该特征在分类中起到的作用越大
clf.feature_importances_

In [ ]:
answer = clf.predict(x_train)
answer2 = clf.predict(x_test)
print(answer)
# print(np.mean( answer == y_train))
print('f1 for train = ' , f1_score(y_train, answer, average='micro'))
print('f1 for test = ' , f1_score(y_test, answer2, average='micro'))


In [ ]:
from sklearn import tree
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import f1_score 

test_mean = []

for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(labels_5, targetArr, test_size = 0.2) 
    #使用信息熵作为划分标准，对决策树进行训练
    clf = tree.DecisionTreeClassifier(criterion='entropy')
    clf.fit(x_train, y_train)
    answer = clf.predict(x_train)
    answer2 = clf.predict(x_test)
    f1_train = f1_score(y_train, answer, average='micro')
    f1_test = f1_score(y_test, answer2, average='micro')
    
    test_mean.append(f1_test)

# print('f1 for train = ' , f1_score(y_train, answer, average='micro'))
# print('f1 for test = ' , f1_score(y_test, answer2, average='micro'))


In [ ]:
## 这里是试验田
import numpy
len(test_mean)

narray = numpy.array(test_mean)

narray.sum() / 100

### 贝叶斯

In [ ]:
from sklearn import linear_model
clf_bayes = linear_model.BayesianRidge()
clf_bayes.fit(x_train, y_train)
answer = clf_bayes.predict(x_train).round() #取整
answer2 = clf_bayes.predict(x_test).round()
print('f1 for train = ' , f1_score(y_train, answer, average='micro'))
print('f1 for test = ' , f1_score(y_test, answer2, average='micro'))

## 聚类方法的研究


In [ ]:
from sklearn.cluster   import KMeans 
#使用默认的K-Means算法  
num_clusters = 4
kmeans_clf = KMeans(n_clusters=num_clusters)  
kmeans_clf.fit(x_train)  
print(kmeans_clf.labels_) 

answer = kmeans_clf.predict(x_train)
answer2 = kmeans_clf.predict(x_test)
print('f1 for train = ' , f1_score(y_train, answer, average='micro'))
print('f1 for test = ' , f1_score(y_test, answer2, average='micro'))